In [31]:
#required libraries
import os
import PIL
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [32]:
# Load CSV data
data_df = pd.read_csv('soil_samples_details1.csv')  


In [33]:
 #Extend CSV data to include all orientations
expanded_data = []
for _, row in data_df.iterrows():
    sample_name = row['Sample Name']  
    for orientation in range(1, 4):  
        img_name = f"{sample_name}_{orientation}"  
        expanded_data.append([img_name, row['N'], row['P'], row['K']])


In [34]:
# Create a new DataFrame with the expanded data
expanded_df = pd.DataFrame(expanded_data, columns=['Sample Name', 'N', 'P', 'K'])
expanded_df['filepath'] = expanded_df['Sample Name'] + '.jpg'  

In [35]:
# Split data into train and validation sets
train_df, val_df = train_test_split(expanded_df, test_size=0.2, random_state=42)

In [36]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,  # Increased rotation for better generalization
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,  
    zoom_range=0.2,  
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = datagen.flow_from_dataframe(
    train_df,
    directory='soil images',  # Path to the folder containing images
    x_col='filepath',
    y_col=['N', 'P', 'K'],
    target_size=(224, 224),
    class_mode='raw',
    batch_size=8,
    shuffle=True
)

val_gen = datagen.flow_from_dataframe(
    val_df,
    directory='soil images',  # Path to the folder containing images
    x_col='filepath',
    y_col=['N', 'P', 'K'],
    target_size=(224, 224),
    class_mode='raw',
    batch_size=8,
    shuffle=False
)

Found 72 validated image filenames.
Found 18 validated image filenames.


In [37]:
# Define the model architecture using ResNet50 as base
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  
output = Dense(3, activation='linear')(x)  # 3 outputs for N, P, K

model = Model(inputs=base_model.input, outputs=output)

In [38]:
# Freeze the base model layers during initial training
for layer in base_model.layers:
    layer.trainable = False


In [ ]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_soil_content_model.keras', save_best_only=True, monitor='val_loss')


# Train the model
epochs = 20
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=[early_stopping, model_checkpoint]
)

d:\soilmodel\enve\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - loss: 113929.2344 - mae: 204.1069 - val_loss: 34273.0469 - val_mae: 123.5327
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - loss: 118103.5547 - mae: 193.6089 - val_loss: 27498.5234 - val_mae: 103.6047
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 82624.7266 - mae: 159.1732 - val_loss: 20219.3633 - val_mae: 84.0609
Epoch 4/20


In [ ]:
# Save the model
model.save("soil_content_model.keras")



In [ ]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("soil_content_model.keras")

d:\soilmodel\enve\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
import cv2 
import numpy as np

def preprocess_image(testimages):
    # Load the image
    img = cv2.imread(testimages)
    
    # Resize the image to the same dimensions as training images (224, 224)
    img_resized = cv2.resize(img, (224, 224))
    
    # Normalize the image to match the preprocessing used during training
    img_normalized = img_resized / 255.0
    
    # Expand dimensions to add a batch dimension (1, 224, 224, 3)
    img_batch = np.expand_dims(img_normalized, axis=0)
    
    return img_batch

In [ ]:
def predict_soil_content(testimages):
    # Preprocess the image
    img_batch = preprocess_image(testimages)
    
    # Make prediction
    prediction = model.predict(img_batch)
    
    predicted_N, predicted_P, predicted_K = prediction[0]
    
    return predicted_N, predicted_P, predicted_K


In [ ]:
# Path to the new test image
test_image_path = "testimages\IMG B1_(1).jpg"  # Replace with your test image path

# Get predictions
predicted_N, predicted_P, predicted_K = predict_soil_content(test_image_path)

# Display results
print(f"Predicted Nitrogen (N): {predicted_N}")
print(f"Predicted Phosphorus (P): {predicted_P}")
print(f"Predicted Potassium (K): {predicted_K}")

<>:2: SyntaxWarning: invalid escape sequence '\I'
<>:2: SyntaxWarning: invalid escape sequence '\I'
C:\Users\jayzzz45\AppData\Local\Temp\ipykernel_47884\1241951237.py:2: SyntaxWarning: invalid escape sequence '\I'
  test_image_path = "testimages\IMG B1_(1).jpg"  # Replace with your test image path


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Nitrogen (N): 151.45901489257812
Predicted Phosphorus (P): 16.67083168029785
Predicted Potassium (K): 230.8568878173828
